In [ ]:
import sys
sys.path.append('..')
from common.time_layers import *
from common.np import *  # import numpy as np
from common.base_model import BaseModel

In [ ]:
class BetterRnnlm(BaseModel):
    def __init__(self,vocab_size=10000,wordvec_size=650,hidden_size=650,dropout_ratio=0.5):
        #重みの初期化
        V,D,H=vocab_size,wordvec_size,hidden_size
        rn=np.random.randn

        embed_W=(rn(V,D)/100).astype('f')
        lstm1_Wx=(rn(D,4*H)/np.sqrt(D)).astype('f')
        lstm1_Wh=(rn(H,4*H)/np.sqrt(H)).astype('f')
        lstm1_b=np.zeros(4*H).astype('f')
        lstm2_Wx=(rn(H,4*H)/np.sqrt(D)).astype('f') #２つ目のLSTMレイヤへの入力は、１つ目のLSTMレイヤの出力hs(N,T,H)のため、重みWxの形状は(H,4*H)
        lstm2_Wh=(rn(H,4*H)/np.sqrt(H)).astype('f')
        lstm2_b=np.zeros(4*H).astype('f')
        affine_b=np.zeros(V).astype('f')

        #レイヤの生成
        self.layers=[
                     TimeEmbedding(embed_W),
                     TimeDropout(dropout_ratio),
                     TimeLSTM(lstm1_Wx,lstm1_Wh,lstm1_b,stateful=True),
                     TimeDropout(dropout_ratio),
                     TimeLSTM(lstm2_Wx,lstm2_Wh,lstm2_b,stateful=True),
                     TimeDropout(dropout_ratio),
                     TimeAffine(embed_W.T,affine_b)   #Embeddingレイヤと重みを共有
        ]
        self.loss_layer=TimeSoftmaxWithLoss()
        self.lstm_layers=[self.layers[2],self.layers[4]]
        self.drop_layers=[self.layers[1],self.layers[3],self.layers[5]]

        #パラメータの格納
        self.params,self.grads=[],[]
        for layer in self.layers:
            self.params+=layer.params
            self.grads+=layer.grads

    def predict(self,xs,train_flg=False):
        for layer in self.drop_layers:
            layer.train_flg=train_flg
        for layer in self.layers:
            xs=layer.forward(xs)
        return xs

    def forward(self,xs,ts,train_flg=True):
        score=self.predict(xs,train_flg)
        loss=self.loss_layer.forward(score,ts)
        return loss

    def backward(self,dout=1):
        dout=self.loss_layer.backward(dout)
        for layer in reversed(self.layers):
            dout=layer.backward(dout)
        return dout

    def reset_state(self):
        for layer in self.lstm_layers:
            layer.reset_state()